---
---

<h1><center><ins>Exercise Sheet 5</ins></center></h1>
<h2><center>Numerical Methods <br>

---
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import exp, cos

## Exercise 1 - Numerical derivative

**(A)** Compute the derivative of the following two functions:

$$ f_1(x) = x^2 + \cos(x) $$

$$ f_2(x) = \exp(x) - x^3 $$

in correspondence of at least $N = 50$ values $x_i$ ($i = 1, ..., N$) of the variable $x$ in the interval $[1,5]$. To do this, *write your own code to compute the derivatives numerically*.

**(B)** Compare the resulting values you get for the numerical derivatives with the corresponding analytic solutions. Calculate the value of the quantity $q$ for each function:

$$ q = \sum_{i=1}^N \frac{\left[d_i - f'(x_i) \right]}{f'(x_i)} $$

where $d_i$ are the values you obtained for the numerical derivatives for each of the $N$ abscissas, and $f'(x_i)$ are the analytical derivatives evaluated in points $x_i$. How does the value of $q$ change when changing the value of $h$ to compute the derivatives numerically? How do the values of $q$ compare for the two functions? Discuss your findings.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


functions = {
    "f_1": {
        "func": lambda x: x**2 + np.cos(x),
        "analytic": lambda x: 2*x - np.sin(x)
    },
    "f_2": {
        "func": lambda x: np.exp(x) - x**3,
        "analytic": lambda x: np.exp(x) - 3*x**2
    }
}

def numerical_derivative(func, x, h=1e-5):
    return (func(x + h) - func(x - h)) / (2.0 * h)

def q_metric(numerical, analytic, eps=0.0):
    analytic = np.asarray(analytic)
    numerical = np.asarray(numerical)
    mask = np.abs(analytic) > eps
    if not np.all(mask):
        n_excluded = np.sum(~mask)
    else:
        n_excluded = 0
    if np.sum(mask) == 0:
        raise ValueError("Error")
    return np.sum((numerical[mask] - analytic[mask]) / analytic[mask]), n_excluded


l = np.linspace(1, 5, 50)


hs = [1e-5, 1e-3]

for name, data in functions.items():
    f = data["func"]
    fprime_analytic = data["analytic"](l)

    
    numericals = {h: numerical_derivative(f, l, h) for h in hs}

    
    for h, d_num in numericals.items():
        q_val, excluded = q_metric(d_num, fprime_analytic, eps=1e-14)
        print(f"{name}: q (h={h}) = {q_val:.6e}  (excluded {excluded} points due to analytic ≈ 0)")

    
    plt.figure(figsize=(8, 5))
    plt.plot(l, f(l), label=f"{name}(x)")
    plt.plot(l, fprime_analytic, '--', label="Analytic derivative")
    for h, d_num in numericals.items():
        plt.plot(l, d_num, ':', label=f"Numerical derivative (h={h})")
    plt.title(f"{name} and derivatives")
    plt.xlabel("x")
    plt.ylabel(f"{name} and derivatives")
    plt.legend()
    plt.tight_layout()
    plt.show()

## Exercise 2 - Newton-Cotes formulas

In the file ```surface_luminosity.txt``` you will find the numerical function describing the surface luminosity $\Sigma$ of a globular star cluster as a function of the distance $R$ from its centre (projected on the plane of the sky). Compute the total luminosity $L$ of the system by performing the integral

$$ L = \int_0^{R_{\text{max}}} \Sigma(R) \, 2 \pi R \, dR $$

using the trapezoid rule and Simpson's rule, and compare the results you obtain. To do this, first choose one of these methods and implement **your own algorithm to compute the integral with it**; for the other method, use the corresponding built-in python function ```scipy.integrate.trapz``` or ```scipy.integrate.simps``` and familiarize on its usage.

In [ ]:
import numpy as np
from scipy import integrate as integr


path = "/workspaces/GLOOMY-COBWEB/surface_luminosity.txt"
r, sl = np.loadtxt(path, comments="#", unpack=True)


def luminosity_integrand(r_val):
    """2π r * surface_luminosity(r)"""
    return 2 * np.pi * r_val * np.interp(r_val, r, sl)

def simpsons_rule(func, a, b, n):
    if n % 2 == 1:
        n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = func(x)
    return (h / 3) * (y[0] + y[-1] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]))

a, b = r[0], r[-1]
n_points = 1000
r_grid = np.linspace(a, b, n_points)
y_grid = luminosity_integrand(r_grid)

L_manual = simpsons_rule(luminosity_integrand, a, b, n_points)
L_scipy_simpson = integr.simpson(y_grid, r_grid)
L_scipy_trap = integr.trapezoid(y_grid, r_grid)

print(f"Manual Simpson's Rule: {L_manual:.6e}")
print(f"SciPy Simpson's Rule:  {L_scipy_simpson:.6e}")
print(f"SciPy Trapezoid Rule:  {L_scipy_trap:.6e}")


## Exercise 3 - Gaussian Quadrature

**(A)** Compute the following integral by using a 4-point Gaussian quadrature. 

$$ I = \int_{-1}^{1} \cos(x) dx $$

To do this, use the Legendre polynomial of degree 4:

$$ P_4(x) = \frac{1}{8}(35 x^4 - 30 x^2 +3) $$

At this link: https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature you can find the roots of Legendre polynomials and the necessary weights to solve the integral (i.e., you do not have to calculate them all from scratch!).

**(B)** Compare the result with the ones you obtain when using a Legendre polynomial of degree 3, 2, and 1, and comment your findings.

In [ ]:
import numpy as np
from scipy import integrate as integr


def f(x):
    return np.cos(x)

gauss_data = {
    1: {"roots": [0.0], "weights": [2.0]},
    2: {"roots": [-0.5773502691896257, 0.5773502691896257],
        "weights": [1.0, 1.0]},
    3: {"roots": [-0.7745966692414834, 0.0, 0.7745966692414834],
        "weights": [0.5555555555555556, 0.8888888888888888, 0.5555555555555556]},
    4: {"roots": [-0.8611363115940526, -0.3399810435848563,
                  0.3399810435848563, 0.8611363115940526],
        "weights": [0.3478548451374538, 0.6521451548625461,
                    0.6521451548625461, 0.3478548451374538]}
}


def gaussian_quadrature(func, roots, weights):
    return np.sum(np.array(weights) * func(np.array(roots)))


print("Gaussian Quadrature for f(x) = cos(x) on [-1, 1]\n")
for n in range(1, 5):
    roots = np.array(gauss_data[n]["roots"])
    weights = np.array(gauss_data[n]["weights"])

    
    L_manual = gaussian_quadrature(f, roots, weights)

    
    L_scipy = integr.fixed_quad(f, -1, 1, n=n)[0]

    print(f"Degree {n}:  Manual = {L_manual:.8f},  SciPy = {L_scipy:.8f},  Difference = {abs(L_manual - L_scipy):.2e}")


L_exact = np.sin(1) - np.sin(-1)  # ∫ cos(x) dx = sin(x)
print(f"\nExact analytical value: {L_exact:.8f}")

## Exercise 4 - Monte Carlo integration 

Consider the following integral:
$$ I = \int_0^1 \cos \left(\frac{\pi x}{2} \right) dx = \frac{2}{\pi} $$

**(A)** Compute the above integral and its variance by using the following Monte Carlo methods:

1. mean value

How do your results compare with the true value of the integral?

**(B)** Plot the behaviour of $I$ and of the variance $\sigma_{I}$ as a function of the number $N$ of points you generate for each of the methods. Consider at least 3 different values of $N$.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def g(x):
    return np.cos(np.pi * x / 2)

true_value = 2 / np.pi


def monte_carlo_mean(func, N):
    samples = np.random.rand(N)
    values = func(samples)
    mean_est = np.mean(values)
    var_est = np.var(values) / N  
    return mean_est, var_est


N0 = 100_000
I_est, var_est = monte_carlo_mean(g, N0)
print(f"Mean value estimate (N={N0}): {I_est:.6f}")
print(f"Variance of estimate: {var_est:.6e}")
print(f"True value of the integral: {true_value:.6f}")


Ns = np.arange(100, 10_001, 100)
I_values, var_values = zip(*(monte_carlo_mean(g, n) for n in Ns))


plt.figure(figsize=(12, 5))


plt.subplot(1, 2, 1)
plt.plot(Ns, I_values, label='Estimated I(N)')
plt.axhline(true_value, color='r', linestyle='--', label='True value 2/π')
plt.xlabel('Number of samples N')
plt.ylabel('Integral estimate I(N)')
plt.title('Monte Carlo Estimate vs N')
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(Ns, var_values, color='orange', label='Variance σ_I²(N)')
plt.xlabel('Number of samples N')
plt.ylabel('Variance σ_I²(N)')
plt.title('Variance vs N')
plt.legend()

plt.tight_layout()
plt.show()


## Exercise 5 - Monte Carlo integration continued (optional)

**(A)** Compute the above integral from Exercise 4 and its variance by using the other Monte Carlo methods covered in the lecture:

2. importance sampling
3. control variates
4. antithetic variates

How do your results compare with the true value of the integral?

**(B)** Plot the behaviour of $I$ and of the variance $\sigma_{I}$ as a function of the number $N$ of points you generate for each of the methods. Consider at least 3 different values of $N$.